In [1]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
import pyproj
# import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import h5py
# import pyhdf
# from pyhdf.SD import SD, SDC
# import re
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from PIL import Image
import scipy.stats as stats
import numpy.ma as ma

In [2]:
qa_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_118062_20170605_20200903_02_T1_QA_PIXEL.TIF')



In [3]:
# Define red, nir, and swir bands
red_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_118062_20170605_20200903_02_T1_SR_B4.TIF')
nir_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_118062_20170605_20200903_02_T1_SR_B5.TIF')
swir_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_118062_20170605_20200903_02_T1_SR_B7.TIF')

In [4]:
# convert to pandas dataframe
red = pd.DataFrame(np.array(red_im))
nir = pd.DataFrame(np.array(nir_im))
swir = pd.DataFrame(np.array(swir_im))

In [5]:
# convert fill value to nans
red=red.replace(0,np.nan)
nir=nir.replace(0,np.nan)
swir=swir.replace(0,np.nan)

In [6]:
# filter through only valid values and replace non-valid values with nans 
mask_red = (red >= 7273) & (red <= 43636) 
mask_nir = (nir >= 7273) & (nir <= 43636)
mask_swir = (swir >= 7273) & (swir <= 43636)

red = red[mask_red]
nir = nir[mask_nir]
swir = swir[mask_swir]

In [7]:
# Multiply by scale factor and add offset
red = (0.0000275*red) - 0.2
nir = (0.0000275*nir) - 0.2
swir = (0.0000275*swir) - 0.2

In [8]:
# DELETE IF NEEDED

red = np.asarray(red)
nir = np.asarray(nir)
swir = np.asarray(swir)

In [9]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# rows

# red_arr = np.delete(red,slice(4000,9999999),0)
# nir_arr = np.delete(nir,slice(4000,9999999),0)
# swir_arr = np.delete(swir,slice(4000,9999999),0)

# # columns

# red = np.delete(red_arr,slice(4000,9999999),1)
# nir = np.delete(nir_arr,slice(4000,9999999),1)
# swir = np.delete(swir_arr,slice(4000,9999999),1)


In [10]:
red = red[1300:2300,3200:4200]
nir = nir[1300:2300,3200:4200]
swir = swir[1300:2300,3200:4200]

In [11]:
# check if shape is same for all
print(red.shape)
print(nir.shape)
print(swir.shape)

(1000, 1000)
(1000, 1000)
(1000, 1000)


In [12]:
# Calculate NDVI and assign to variable
ndvi=(nir-red)/(nir+red)

In [13]:
# Delete
ndvi_arr = np.asarray(ndvi) 


In [14]:
# filter ndvi values
ndvi_arr[ndvi_arr < 0] = np.nan
ndvi_arr[ndvi_arr > 1] = np.nan 

In [15]:
ndvi = pd.DataFrame(ndvi_arr)

In [16]:
# convert swir back to array
swir_arr = np.asarray(swir)

In [17]:
# filter swir values
swir_arr[swir_arr < 0.01] = np.nan
swir_arr[swir_arr > 100] = np.nan

In [18]:
swir = pd.DataFrame(swir_arr)

In [19]:
# Calculate STR and assign to variable
str0=((1-swir)**2)/(2*swir)

In [20]:
# THIS IS WHAT TO DO FOR FILTERING SWIR
SWIR=pd.DataFrame(np.asarray(swir).flatten())

In [21]:
SWIR=SWIR.rename(columns={0:'SWIR'})

In [22]:
# END WHAT TO DO FOR FILTERING FOR SWIR

In [23]:
# START WHAT TO DO FOR FILTERING FOR NDVI

In [24]:
NDVI=pd.DataFrame(np.asarray(ndvi).flatten())

In [25]:
NDVI=NDVI.rename(columns={0:'NDVI'})

In [26]:
# (DELETE IF NEEDED)
NDVI['STR'] = ((1-SWIR['SWIR'])**2)/(2*SWIR['SWIR']) 

In [27]:
# MAIN DATAFRAME (DELETE IF NEEDED)
DF = NDVI
DF

,NDVI,STR
0,0.503493,2.573101
1,0.633755,5.259874
2,0.683905,3.776370
3,0.638843,3.577812
4,0.671939,3.351186
...,...,...
999995,0.828455,9.197515
999996,0.820338,9.406140
999997,0.836714,9.157924
999998,0.847933,8.953695


In [28]:
x_wet=np.linspace(0,1,100)
x_dry=np.linspace(0,1,100)

In [29]:
# convert to array and flatten ndvi and str
ndvi_flat=np.asarray(ndvi).flatten()
str0_flat=np.asarray(str0).flatten()

In [30]:
# convert both to list
ndvi_list = ndvi_flat.tolist()
str0_list = str0_flat.tolist()

In [31]:
# calculate STR max list (change "before" equation below accordingly)
str_max_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(30.134*ndvi_list[i])+5.856
            str_max_list.append(before)
    else:
        str_max_list.append(np.nan) 

In [32]:
# calculate STR min list (change "before" equation below accordingly)
str_min_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(2.121*ndvi_list[i])-0.296
            str_min_list.append(before)
    else:
        str_min_list.append(np.nan)

In [33]:
str0_max_arr = np.asarray(str_max_list)
str0_min_arr = np.asarray(str_min_list)

In [34]:
str_max = str0_max_arr.reshape((red.shape[0], red.shape[1]))
str_min = str0_min_arr.reshape((red.shape[0], red.shape[1]))

In [35]:
# plt.figure(figsize=(6,4))
# plt.title('2018-01-31',fontsize=16)
# plt.scatter(DF['NDVI'],DF['STR'],color='black',s=0.01)
# # plt.ylim(-1,100)
# # plt.xlim(0,1)
# plt.xlabel('NDVI',fontsize=14)
# plt.ylabel('STR',fontsize=14)
# plt.tick_params(axis='x', labelsize=12)
# plt.tick_params(axis='y', labelsize=12)
# # plt.scatter(DF['NDVI'],str0_max_arr,color='hotpink')
# # plt.scatter(DF['NDVI'],str0_min_arr,color='green')
# plt.plot(x_wet,(30.134*x_wet+5.856),color='blue',linewidth=3)
# plt.plot(x_dry,(2.121*x_dry)-.296,color='red',linewidth=3)

# plt.savefig('/Users/nikaankoupaei/Downloads/STR_NDVI_PANEL4.png',dpi=300,bbox_inches="tight")

In [36]:
qa_arr = np.asarray(qa_im)

In [37]:
qa_arr_sliced = qa_arr[1300:2300,3200:4200]

In [38]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# rows

# qa_arr = np.delete(qa,slice(4000,9999999),0)

# columns

# qa_arr = np.delete(qa_arr,slice(4000,9999999),1)


In [39]:
qa_arr_sliced.shape

(1000, 1000)

In [40]:
qa = pd.DataFrame(qa_arr_sliced.flatten())

In [41]:
qa.shape

(1000000, 1)

In [42]:
df_qa=qa.rename(columns={0:'QA'})
df_qa

,QA
0,21824
1,21824
2,21824
3,21824
4,21824
...,...
999995,21824
999996,21824
999997,21824
999998,21824


In [43]:
OPTRAM = (str0 - str_min)/(str_max - str_min)

In [44]:
df_OPTRAM = pd.DataFrame(np.asarray(OPTRAM).flatten())

In [45]:
df_OPTRAM=df_OPTRAM.rename(columns={0:'OPTRAM'})
df_OPTRAM

,OPTRAM
0,0.088920
1,0.176181
2,0.103587
3,0.104742
4,0.088969
...,...
999995,0.263504
999996,0.273313
999997,0.259514
999998,0.249162


In [46]:
print(df_qa.shape)
print(df_OPTRAM.shape)

(1000000, 1)
(1000000, 1)


In [47]:
df_OPTRAM['QA'] = df_qa['QA']
df_OPTRAM

,OPTRAM,QA
0,0.088920,21824
1,0.176181,21824
2,0.103587,21824
3,0.104742,21824
4,0.088969,21824
...,...,...
999995,0.263504,21824
999996,0.273313,21824
999997,0.259514,21824
999998,0.249162,21824


In [48]:
# Setting all rows that are not clear (i.e. those that have cloudy pixels)

df_clear = df_OPTRAM.where(df_OPTRAM['QA'] == 21824)

In [49]:
df_clear

,OPTRAM,QA
0,0.088920,21824.0
1,0.176181,21824.0
2,0.103587,21824.0
3,0.104742,21824.0
4,0.088969,21824.0
...,...,...
999995,0.263504,21824.0
999996,0.273313,21824.0
999997,0.259514,21824.0
999998,0.249162,21824.0


In [50]:
OPTRAM_clear_array = np.asarray(df_clear['OPTRAM'])
OPTRAM_clear_array.shape

(1000000,)

In [51]:
OPTRAM_FINAL_CLOUD_MASKED = OPTRAM_clear_array.reshape((1000,1000))
OPTRAM_FINAL_CLOUD_MASKED.shape

(1000, 1000)

In [52]:
pd.DataFrame(OPTRAM_FINAL_CLOUD_MASKED).to_csv('/Users/nikaankoupaei/Downloads/ownCloud/20231222_OPTRAM_FILES_REALIZED_DIDNT_INLCUDE_VALID_RANGE_ENDS_FOR_LANDSAT/HIRANO_DF/NEW_FOR_20231222_cloud_masked_OPTRAM_20170605_HIRANO_DF_NEW_LOCATION_filtered.csv')


